In [30]:
# import necessary BuildingMOTIF libraries
from rdflib import Namespace
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
from buildingmotif.namespaces import BRICK # import this to make writing URIs easier
import rdflib

In [31]:
# Create an in-memory BuildingMOTIF instance
bm = BuildingMOTIF("sqlite://")

In [32]:
# create the namespace for the building
BLDG = Namespace('urn:bldg/')

In [33]:
# create the building model and add a description
model = Model.create(BLDG, description="This is a test model for a simple building")

In [34]:
# load the BRICK model you created
model.graph.parse("BRICK_model_you_just_created.ttl", format="ttl")

<Graph identifier=394f3b01-7301-4884-b0d0-88e17192dc09 (<class 'rdflib.graph.Graph'>)>

In [35]:
# load the ".ttl" constraints shape graphs, one ".ttl" file at a time
# note that the BRICK constraint file "Brick-subset.ttl" is always required. 
# Brick "Brick-subset.ttl" is a subset of BRICK constraint rules, and Brick-full.ttl" is all BRICK constraint rules.
# Normally, "Brick-full.ttl" should have been used for the required BRICK constraint file instead of Brick-subset.ttl". 
# In fact, you should use "Brick-full.ttl" as the required BRICK constraint file in your future BRICK application. 
# However, since BuildingMOTIF is not yet mature, plus for the sake of demonstration below, "Brick-subset.ttl" is used.
brick= Library.load(ontology_graph="Brick-subset.ttl")
constraint1 = Library.load(ontology_graph="constraint1.ttl")
constraint2 = Library.load(ontology_graph="constraint2.ttl")

In [36]:
# or, you have the option to load all the ".ttl" files in a file folder
constraint_file_folder1 = Library.load(directory="constraints_main_folder/constraints_group1")
constraint_file_folder2 = Library.load(directory="constraints_main_folder/constraints_group2")

In [37]:
# gather all the ".ttl" constraints shape graphs in one place
all_shape_graphs = [
    brick.get_shape_collection(),
    constraint1.get_shape_collection(),
    constraint2.get_shape_collection(),
    constraint_file_folder1.get_shape_collection(),
    constraint_file_folder2.get_shape_collection()
]

In [38]:
# view the content of the BRICK model
print(model.graph.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

<urn:bldg/> a owl:Ontology .

<urn:bldg/Core_ZN-PSC_AC> a brick:AHU ;
    brick:hasPart <urn:bldg/Core_ZN-PSC_AC-Clg_Coil>,
        <urn:bldg/Core_ZN-PSC_AC-Damper>,
        <urn:bldg/Core_ZN-PSC_AC-Fan> .

<urn:bldg/Core_ZN-PSC_AC-Clg_Coil> a brick:Cooling_Coil .

<urn:bldg/Core_ZN-PSC_AC-Damper> a brick:Damper .

<urn:bldg/Core_ZN-PSC_AC-Fan> a brick:Supply_Fan .




In [39]:
# The content of the BRICK model should look like this:
"""
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

<urn:bldg/> a owl:Ontology .

<urn:bldg/Core_ZN-PSC_AC> a brick:AHU ;
    brick:hasPart <urn:bldg/Core_ZN-PSC_AC-Clg_Coil>,
        <urn:bldg/Core_ZN-PSC_AC-Damper>,
        <urn:bldg/Core_ZN-PSC_AC-Fan> .

<urn:bldg/Core_ZN-PSC_AC-Clg_Coil> a brick:Cooling_Coil .

<urn:bldg/Core_ZN-PSC_AC-Damper> a brick:Damper .

<urn:bldg/Core_ZN-PSC_AC-Fan> a brick:Supply_Fan .

"""

'\n@prefix brick: <https://brickschema.org/schema/Brick#> .\n@prefix owl: <http://www.w3.org/2002/07/owl#> .\n\n<urn:bldg/> a owl:Ontology .\n\n<urn:bldg/Core_ZN-PSC_AC> a brick:AHU ;\n    brick:hasPart <urn:bldg/Core_ZN-PSC_AC-Clg_Coil>,\n        <urn:bldg/Core_ZN-PSC_AC-Damper>,\n        <urn:bldg/Core_ZN-PSC_AC-Fan> .\n\n<urn:bldg/Core_ZN-PSC_AC-Clg_Coil> a brick:Cooling_Coil .\n\n<urn:bldg/Core_ZN-PSC_AC-Damper> a brick:Damper .\n\n<urn:bldg/Core_ZN-PSC_AC-Fan> a brick:Supply_Fan .\n\n'

In [40]:
# display validation results in more detail

validation_result = model.validate(all_shape_graphs)
print(f"Model is valid? {validation_result.valid}")
print("-"*79) # just a separator for better error display
print(validation_result.report_string)
print("-"*79) # just a separator for better error display
print("Model is invalid for these reasons:")
for diff in validation_result.diffset:
    print(f" - {diff.reason()}")

Model is valid? False
-------------------------------------------------------------------------------
Validation Report
Conforms: False
Results (3):
Constraint Violation in ConstraintComponent (https://nrel.gov/BuildingMOTIF/constraints#countConstraintComponent):
	Severity: sh:Violation
	Source Shape: <urn:my_site_constraints/htg-coil-count>
	Focus Node: <urn:my_site_constraints/>
	Value Node: <urn:my_site_constraints/>
	Message: Not the right number {$exactCount} of instances of {$class}
	Message: need 1 heating coil
Constraint Violation in QualifiedValueShapeConstraintComponent (http://www.w3.org/ns/shacl#QualifiedMinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:path brick:hasPart ; sh:qualifiedMaxCount Literal("1", datatype=xsd:integer) ; sh:qualifiedMinCount Literal("1", datatype=xsd:integer) ; sh:qualifiedValueShape [ sh:node <urn:ashrae/g36/4.8/sz-vav-ahu/sa-fan> ] ]
	Focus Node: <urn:bldg/Core_ZN-PSC_AC>
	Result Path: brick:hasPart
Constraint Violation i

In [41]:
# From the above validation report, the first error is that the BRICK model doesn't have an instance of a heating coil.
# The second error is that the Air Handline Unit (AHU), whose name is "Core_ZN-PSC_AC","hasPart" a supply fan named "Core_ZN-PSC_AC-Fan".
# However, this hasPart-supply-fan relationship doesn't meet the metadata requirements of ASHARE Guideline 36.

In [42]:
# Now, we want to solve the first error first -- adding a heating coil instance.
# In BuildingMOTIF, there are no set rules to correct the validation errors. 
# Thus, one need to examine closely the BRICK model and the validation report.
# For example, the BRICK model shows that an AHU "hasPart" a cooling coil, a damper, and a supply fan.
# Particularly, notice how each piece of equipment is named. The AHU is "Core_ZN-PSC_AC". 
# The cooling coil, which is part of the AHU, is named "Core_ZN-PSC_AC-Clg_Coil". 
# Thus, when adding the heating coil, one needs to follow closely this naming convention.
# The process of adding a heating coil is shown below.



# list the name of the AHU
ahu_name = "Core_ZN-PSC_AC"

# add the name and the brick class of the heating coil to the BRICK model
equipment_name = f"{ahu_name}-Htg_Coil"
equipment_brick_class="Heating_Coil" 
dummy_code = f"equipment_template= brick.get_template_by_name(BRICK.{equipment_brick_class})"
exec(dummy_code)
equipment_binding_dict = {"name": BLDG[equipment_name]}
equipment_graph = equipment_template.evaluate(equipment_binding_dict)
model.add_graph(equipment_graph)

# add "hasPart" relationship between the AHU and the heating coil
BRICK_relation="hasPart"
dummy_code=f"model.graph.add((BLDG['{ahu_name}'], BRICK.{BRICK_relation}, BLDG['{equipment_name}']))"
exec(dummy_code)


# There might be a confusion between "model.add_graph()" and "model.graph.add()"
# "model.add_graph()" is adding the equipment name and the brick class 
# "model.graph.add()" is adding the "hasPart", "hasPoint", or "feeds" relationship between 2 pieces of equipment


# print the resulting BRICK model to confirm that the heating coil was added and connected
print(model.graph.serialize())


@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

<urn:bldg/> a owl:Ontology .

<urn:bldg/Core_ZN-PSC_AC> a brick:AHU ;
    brick:hasPart <urn:bldg/Core_ZN-PSC_AC-Clg_Coil>,
        <urn:bldg/Core_ZN-PSC_AC-Damper>,
        <urn:bldg/Core_ZN-PSC_AC-Fan>,
        <urn:bldg/Core_ZN-PSC_AC-Htg_Coil> .

<urn:bldg/Core_ZN-PSC_AC-Clg_Coil> a brick:Cooling_Coil .

<urn:bldg/Core_ZN-PSC_AC-Damper> a brick:Damper .

<urn:bldg/Core_ZN-PSC_AC-Fan> a brick:Supply_Fan .

<urn:bldg/Core_ZN-PSC_AC-Htg_Coil> a brick:Heating_Coil .




In [43]:
# display validation results in more detail

validation_result = model.validate(all_shape_graphs)
print(f"Model is valid? {validation_result.valid}")
print("-"*79) # just a separator for better error display
print(validation_result.report_string)
print("-"*79) # just a separator for better error display
print("Model is invalid for these reasons:")
for diff in validation_result.diffset:
    print(f" - {diff.reason()}")

Model is valid? False
-------------------------------------------------------------------------------
Validation Report
Conforms: False
Results (2):
Constraint Violation in QualifiedValueShapeConstraintComponent (http://www.w3.org/ns/shacl#QualifiedMinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:path brick:hasPart ; sh:qualifiedMaxCount Literal("1", datatype=xsd:integer) ; sh:qualifiedMinCount Literal("1", datatype=xsd:integer) ; sh:qualifiedValueShape [ sh:node <urn:ashrae/g36/4.8/sz-vav-ahu/sa-fan> ] ]
	Focus Node: <urn:bldg/Core_ZN-PSC_AC>
	Result Path: brick:hasPart
Constraint Violation in QualifiedValueShapeConstraintComponent (http://www.w3.org/ns/shacl#QualifiedMinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:path brick:hasPart ; sh:qualifiedMaxCount Literal("1", datatype=xsd:integer) ; sh:qualifiedMinCount Literal("1", datatype=xsd:integer) ; sh:qualifiedValueShape [ sh:node <urn:ashrae/g36/4.8/sz-vav-ahu/sa-fan> ] ]
	Focus Nod

In [44]:
# Now, we will focus on correcting the second error -- adding metadata requirements of ASHARE Guideline 36.
# BuildingMOTIF can help generate ".ttl" templates to be added to the existing BRICK model, so that the BRICK model will pass validation.

In [45]:
# We first create a new template library by adding any name you like to the new template library. 
# This new template library is stored in the "generated_templates" variable
generated_templates = Library.create("any_template_library_name_you_like")

In [46]:
# We then use the following code to auto-generate ".ttl" templates and add them to the "generated_templates" variable. 
# For each failed validation item "diff", one template is auto-generated
for diff in validation_result.diffset:
    diff.resolve(generated_templates)

In [47]:
# Print the auto-generated templates. For each auto-generated template, the template name, the template parameters, and the template body are printed. 
for template in generated_templates.get_templates():
    template = template.inline_dependencies()
    print(f"Name (autogenerated): {template.name}")
    print(f"Parameters (autogenerated): {template.parameters}")
    print("Template body (autogenerated):")
    print(template.body.serialize())
    print("-"*79) # just a separator for better display

Name (autogenerated): resolve193285d6
Parameters (autogenerated): {'p25', 'name', 'p27', 'p26', 'p29', 'p28', 'p30'}
Template body (autogenerated):
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix vav48: <urn:ashrae/g36/4.8/sz-vav-ahu/> .

<urn:bldg/Core_ZN-PSC_AC> brick:hasPart <urn:___param___#name> .

<urn:___param___#name> a brick:Supply_Fan,
        vav48:sa-fan ;
    brick:hasPoint <urn:___param___#p25>,
        <urn:___param___#p26>,
        <urn:___param___#p27>,
        <urn:___param___#p28>,
        <urn:___param___#p29>,
        <urn:___param___#p30> .

<urn:___param___#p25> a brick:Fan_Status .

<urn:___param___#p26> a brick:Fan_Status .

<urn:___param___#p27> a brick:Start_Stop_Command .

<urn:___param___#p28> a brick:Frequency_Command .

<urn:___param___#p29> a brick:Start_Stop_Command .

<urn:___param___#p30> a brick:Frequency_Command .


-------------------------------------------------------------------------------
Name (autogenerated): resolve886a5c25


In [48]:
# The printed template should look like this:

"""
Name (autogenerated): resolve81c2f088
Parameters (autogenerated): {'p5', 'p6', 'name', 'p4'}
Template body (autogenerated):
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix vav48: <urn:ashrae/g36/4.8/sz-vav-ahu/> .

<urn:bldg/Core_ZN-PSC_AC> brick:hasPart <urn:___param___#name> .

<urn:___param___#name> a brick:Supply_Fan,
        vav48:sa-fan ;
    brick:hasPoint <urn:___param___#p4>,
        <urn:___param___#p5>,
        <urn:___param___#p6> .

<urn:___param___#p4> a brick:Start_Stop_Command .

<urn:___param___#p5> a brick:Fan_Status .

<urn:___param___#p6> a brick:Frequency_Command .


-------------------------------------------------------------------------------
"""

"\nName (autogenerated): resolve81c2f088\nParameters (autogenerated): {'p5', 'p6', 'name', 'p4'}\nTemplate body (autogenerated):\n@prefix brick: <https://brickschema.org/schema/Brick#> .\n@prefix vav48: <urn:ashrae/g36/4.8/sz-vav-ahu/> .\n\n<urn:bldg/Core_ZN-PSC_AC> brick:hasPart <urn:___param___#name> .\n\n<urn:___param___#name> a brick:Supply_Fan,\n        vav48:sa-fan ;\n    brick:hasPoint <urn:___param___#p4>,\n        <urn:___param___#p5>,\n        <urn:___param___#p6> .\n\n<urn:___param___#p4> a brick:Start_Stop_Command .\n\n<urn:___param___#p5> a brick:Fan_Status .\n\n<urn:___param___#p6> a brick:Frequency_Command .\n\n\n-------------------------------------------------------------------------------\n"

In [49]:
# We notice that the generated template has some undefined names: "<urn:___param___#name>", "<urn:___param___#p4>", "<urn:___param___#p5>", and "<urn:___param___#p6>".
# These undefined names correspond to "Parameters (autogenerated): {'p5', 'p6', 'name', 'p4'}", which summarizes how many items are missing names.
# If we now directly add this generated template (with undefined names) to the Brick model:

for template in generated_templates.get_templates():
    template = template.inline_dependencies()
    model.add_graph(template.body)
print(model.graph.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix vav48: <urn:ashrae/g36/4.8/sz-vav-ahu/> .

<urn:bldg/> a owl:Ontology .

<urn:bldg/Core_ZN-PSC_AC> a brick:AHU ;
    brick:hasPart <urn:___param___#name>,
        <urn:bldg/Core_ZN-PSC_AC-Clg_Coil>,
        <urn:bldg/Core_ZN-PSC_AC-Damper>,
        <urn:bldg/Core_ZN-PSC_AC-Fan>,
        <urn:bldg/Core_ZN-PSC_AC-Htg_Coil> .

<urn:___param___#name> a brick:Supply_Fan,
        vav48:sa-fan ;
    brick:hasPoint <urn:___param___#p19>,
        <urn:___param___#p20>,
        <urn:___param___#p21>,
        <urn:___param___#p22>,
        <urn:___param___#p23>,
        <urn:___param___#p24>,
        <urn:___param___#p25>,
        <urn:___param___#p26>,
        <urn:___param___#p27>,
        <urn:___param___#p28>,
        <urn:___param___#p29>,
        <urn:___param___#p30> .

<urn:___param___#p19> a brick:Fan_Status .

<urn:___param___#p20> a brick:Fan_Status .

<urn:___param___#p21> 

In [50]:
# The resulting Brick model will look like this:


"""

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix vav48: <urn:ashrae/g36/4.8/sz-vav-ahu/> .

<urn:bldg/> a owl:Ontology .

<urn:bldg/Core_ZN-PSC_AC> a brick:AHU ;
    brick:hasPart <urn:___param___#name>,
        <urn:bldg/Core_ZN-PSC_AC-Clg_Coil>,
        <urn:bldg/Core_ZN-PSC_AC-Damper>,
        <urn:bldg/Core_ZN-PSC_AC-Fan>,
        <urn:bldg/Core_ZN-PSC_AC-Htg_Coil> .

<urn:___param___#name> a brick:Supply_Fan,
        vav48:sa-fan ;
    brick:hasPoint <urn:___param___#p4>,
        <urn:___param___#p5>,
        <urn:___param___#p6> .

<urn:___param___#p4> a brick:Start_Stop_Command .

<urn:___param___#p5> a brick:Frequency_Command .

<urn:___param___#p6> a brick:Fan_Status .

<urn:bldg/Core_ZN-PSC_AC-Clg_Coil> a brick:Cooling_Coil .

<urn:bldg/Core_ZN-PSC_AC-Damper> a brick:Damper .

<urn:bldg/Core_ZN-PSC_AC-Fan> a brick:Supply_Fan .

<urn:bldg/Core_ZN-PSC_AC-Htg_Coil> a brick:Heating_Coil .


"""

'\n\n@prefix brick: <https://brickschema.org/schema/Brick#> .\n@prefix owl: <http://www.w3.org/2002/07/owl#> .\n@prefix vav48: <urn:ashrae/g36/4.8/sz-vav-ahu/> .\n\n<urn:bldg/> a owl:Ontology .\n\n<urn:bldg/Core_ZN-PSC_AC> a brick:AHU ;\n    brick:hasPart <urn:___param___#name>,\n        <urn:bldg/Core_ZN-PSC_AC-Clg_Coil>,\n        <urn:bldg/Core_ZN-PSC_AC-Damper>,\n        <urn:bldg/Core_ZN-PSC_AC-Fan>,\n        <urn:bldg/Core_ZN-PSC_AC-Htg_Coil> .\n\n<urn:___param___#name> a brick:Supply_Fan,\n        vav48:sa-fan ;\n    brick:hasPoint <urn:___param___#p4>,\n        <urn:___param___#p5>,\n        <urn:___param___#p6> .\n\n<urn:___param___#p4> a brick:Start_Stop_Command .\n\n<urn:___param___#p5> a brick:Frequency_Command .\n\n<urn:___param___#p6> a brick:Fan_Status .\n\n<urn:bldg/Core_ZN-PSC_AC-Clg_Coil> a brick:Cooling_Coil .\n\n<urn:bldg/Core_ZN-PSC_AC-Damper> a brick:Damper .\n\n<urn:bldg/Core_ZN-PSC_AC-Fan> a brick:Supply_Fan .\n\n<urn:bldg/Core_ZN-PSC_AC-Htg_Coil> a brick:Heating

In [51]:
# Now, if we pass the Brick model to validation, it will still cause validation error:

validation_result = model.validate(all_shape_graphs)
print(f"Model is valid? {validation_result.valid}")
print("-"*79) # just a separator for better error display
print(validation_result.report_string)
print("-"*79) # just a separator for better error display
print("Model is invalid for these reasons:")
for diff in validation_result.diffset:
    print(f" - {diff.reason()}")

Model is valid? False
-------------------------------------------------------------------------------
Validation Report
Conforms: False
Results (3):
Constraint Violation in ConstraintComponent (https://nrel.gov/BuildingMOTIF/constraints#countConstraintComponent):
	Severity: sh:Violation
	Source Shape: <urn:my_site_constraints/fan-count>
	Focus Node: <urn:my_site_constraints/>
	Value Node: <urn:my_site_constraints/>
	Message: Not the right number {$exactCount} of instances of {$class}
	Message: need 1 supply fan
Constraint Violation in QualifiedValueShapeConstraintComponent (http://www.w3.org/ns/shacl#QualifiedMinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:path brick:hasPart ; sh:qualifiedMaxCount Literal("1", datatype=xsd:integer) ; sh:qualifiedMinCount Literal("1", datatype=xsd:integer) ; sh:qualifiedValueShape [ sh:node <urn:ashrae/g36/4.8/sz-vav-ahu/sa-fan> ] ]
	Focus Node: <urn:bldg/Core_ZN-PSC_AC>
	Result Path: brick:hasPart
Constraint Violation in Quali

In [52]:
# Why so? That is because the original Brick model already has the supply fan "<urn:bldg/Core_ZN-PSC_AC-Fan> a brick:Supply_Fan ."
# The template tries to add the supply fan "<urn:___param___#name> a brick:Supply_Fan"
# But since the newly added supply fan name is undefined, the Brick model (with the generated template added) now has 2 supply fans.
# This doesn't meet the validation requirement that the AHU can only have between 1 and 1 instances of supply fan. 
# Thus, the solution is that after generating the template, we need to change the supply fan name from "___param___#name" to "Core_ZN-PSC_AC-Fan"
# before adding the template to the Brick model, so that when the supply fan from the generated template is added to the original Brick model,
# the Brick model can see that the added supply fan from the generated template is the same as the supply fan already existing in the original Brick model,
# thus automatically deleting duplicates.


In [53]:
# The BuildingMOTIF codes to add to undefined names are shown below:


from buildingmotif.namespaces import BRICK
for template in generated_templates.get_templates():
    template = template.inline_dependencies()
    
    # get name of AHU 
    ahu_name = 'urn:bldg/Core_ZN-PSC_AC'
    ahu_name = rdflib.term.URIRef(ahu_name) # necessary conversion from strings to rdflib URI reference

    # get name of the supply fan
    supply_fan_name = ahu_name + '-Fan'

    # we know from the exploration above that each template has
    # 1 parameter called 'name', which we assign to the name of the supply fan
    bindings = {
        "name": supply_fan_name,
    }
    
    # the rest of the parameters (p4, p5, p6) have random names, so let's generate
    # random URIs for those by putting the name of the parameter at the end
    # of the fan's name
    for p in template.parameters - {'name'}: # template.parameters = {'name', 'p6', 'p4', 'p5'}
        print(p)
        bindings[p] = supply_fan_name + p
        bindings[p]=rdflib.term.URIRef(bindings[p]) # necessary conversion from strings to rdflib URI reference
    
    print(bindings)
    """
    bindings =
    
    {'name': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fan'), 
    'p5': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fanp5'), 
    'p4': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fanp4'), 
    'p6': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fanp6')
    }
    
    """

    
    # Execute name change and store it in a new template "template_with_defined_names"
    template_with_defined_names = template.evaluate(bindings)
    
    """
    Basically the name changes look like these
    
    <urn:___param___#name> ==> <urn:bldg/Core_ZN-PSC_AC-Fan>
    <urn:___param___#p4> ==> <urn:bldg/Core_ZN-PSC_AC-Fanp4>
    <urn:___param___#p5> ==> <urn:bldg/Core_ZN-PSC_AC-Fanp5> 
    <urn:___param___#p6> ==> <urn:bldg/Core_ZN-PSC_AC-Fanp6>
    
    """
    
    model.add_graph(template_with_defined_names)


p25
p27
p26
p29
p28
p30
{'name': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fan'), 'p25': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fanp25'), 'p27': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fanp27'), 'p26': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fanp26'), 'p29': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fanp29'), 'p28': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fanp28'), 'p30': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fanp30')}
p21
p22
p23
p24
p19
p20
{'name': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fan'), 'p21': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fanp21'), 'p22': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fanp22'), 'p23': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fanp23'), 'p24': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fanp24'), 'p19': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fanp19'), 'p20': rdflib.term.URIRef('urn:bldg/Core_ZN-PSC_AC-Fanp20')}


In [54]:
# print the resulting BRICK model
print(model.graph.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix vav48: <urn:ashrae/g36/4.8/sz-vav-ahu/> .

<urn:bldg/> a owl:Ontology .

<urn:bldg/Core_ZN-PSC_AC> a brick:AHU ;
    brick:hasPart <urn:___param___#name>,
        <urn:bldg/Core_ZN-PSC_AC-Clg_Coil>,
        <urn:bldg/Core_ZN-PSC_AC-Damper>,
        <urn:bldg/Core_ZN-PSC_AC-Fan>,
        <urn:bldg/Core_ZN-PSC_AC-Htg_Coil> .

<urn:___param___#name> a brick:Supply_Fan,
        vav48:sa-fan ;
    brick:hasPoint <urn:___param___#p19>,
        <urn:___param___#p20>,
        <urn:___param___#p21>,
        <urn:___param___#p22>,
        <urn:___param___#p23>,
        <urn:___param___#p24>,
        <urn:___param___#p25>,
        <urn:___param___#p26>,
        <urn:___param___#p27>,
        <urn:___param___#p28>,
        <urn:___param___#p29>,
        <urn:___param___#p30> .

<urn:___param___#p19> a brick:Fan_Status .

<urn:___param___#p20> a brick:Fan_Status .

<urn:___param___#p21> 

In [55]:
# The resulting Brick model should look like this:


"""
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix vav48: <urn:ashrae/g36/4.8/sz-vav-ahu/> .

<urn:bldg/> a owl:Ontology .

<urn:bldg/Core_ZN-PSC_AC> a brick:AHU ;
    brick:hasPart <urn:___param___#name>,
        <urn:bldg/Core_ZN-PSC_AC-Clg_Coil>,
        <urn:bldg/Core_ZN-PSC_AC-Damper>,
        <urn:bldg/Core_ZN-PSC_AC-Fan>,
        <urn:bldg/Core_ZN-PSC_AC-Htg_Coil> .

<urn:___param___#name> a brick:Supply_Fan,
        vav48:sa-fan ;
    brick:hasPoint <urn:___param___#p4>,
        <urn:___param___#p5>,
        <urn:___param___#p6> .

<urn:___param___#p4> a brick:Start_Stop_Command .

<urn:___param___#p5> a brick:Frequency_Command .

<urn:___param___#p6> a brick:Fan_Status .

<urn:bldg/Core_ZN-PSC_AC-Clg_Coil> a brick:Cooling_Coil .

<urn:bldg/Core_ZN-PSC_AC-Damper> a brick:Damper .

<urn:bldg/Core_ZN-PSC_AC-Fan> a brick:Supply_Fan,
        vav48:sa-fan ;
    brick:hasPoint <urn:bldg/Core_ZN-PSC_AC-Fanp4>,
        <urn:bldg/Core_ZN-PSC_AC-Fanp5>,
        <urn:bldg/Core_ZN-PSC_AC-Fanp6> .

<urn:bldg/Core_ZN-PSC_AC-Fanp4> a brick:Start_Stop_Command .

<urn:bldg/Core_ZN-PSC_AC-Fanp5> a brick:Frequency_Command .

<urn:bldg/Core_ZN-PSC_AC-Fanp6> a brick:Fan_Status .

<urn:bldg/Core_ZN-PSC_AC-Htg_Coil> a brick:Heating_Coil .
"""

'\n@prefix brick: <https://brickschema.org/schema/Brick#> .\n@prefix owl: <http://www.w3.org/2002/07/owl#> .\n@prefix vav48: <urn:ashrae/g36/4.8/sz-vav-ahu/> .\n\n<urn:bldg/> a owl:Ontology .\n\n<urn:bldg/Core_ZN-PSC_AC> a brick:AHU ;\n    brick:hasPart <urn:___param___#name>,\n        <urn:bldg/Core_ZN-PSC_AC-Clg_Coil>,\n        <urn:bldg/Core_ZN-PSC_AC-Damper>,\n        <urn:bldg/Core_ZN-PSC_AC-Fan>,\n        <urn:bldg/Core_ZN-PSC_AC-Htg_Coil> .\n\n<urn:___param___#name> a brick:Supply_Fan,\n        vav48:sa-fan ;\n    brick:hasPoint <urn:___param___#p4>,\n        <urn:___param___#p5>,\n        <urn:___param___#p6> .\n\n<urn:___param___#p4> a brick:Start_Stop_Command .\n\n<urn:___param___#p5> a brick:Frequency_Command .\n\n<urn:___param___#p6> a brick:Fan_Status .\n\n<urn:bldg/Core_ZN-PSC_AC-Clg_Coil> a brick:Cooling_Coil .\n\n<urn:bldg/Core_ZN-PSC_AC-Damper> a brick:Damper .\n\n<urn:bldg/Core_ZN-PSC_AC-Fan> a brick:Supply_Fan,\n        vav48:sa-fan ;\n    brick:hasPoint <urn:bldg/Co

In [56]:
# We notice that our previous mistake of adding the template with undefined names is still showing in the newly resulting Brick model
# While a "partial delete" function might or might not exist in BuildingMOTIF, there is an easy way:
# copy the above Brick model, manually delete all the code lines with undefined names, and store it in a new file 
# The new file can be named "updated_BRICK_model_you_just_created.ttl"



with open("updated_BRICK_model_you_just_created.ttl", "w") as f:
    f.write("""
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix vav48: <urn:ashrae/g36/4.8/sz-vav-ahu/> .

<urn:bldg/> a owl:Ontology .

<urn:bldg/Core_ZN-PSC_AC> a brick:AHU ;
    brick:hasPart <urn:bldg/Core_ZN-PSC_AC-Clg_Coil>,
        <urn:bldg/Core_ZN-PSC_AC-Damper>,
        <urn:bldg/Core_ZN-PSC_AC-Fan>,
        <urn:bldg/Core_ZN-PSC_AC-Htg_Coil> .



<urn:bldg/Core_ZN-PSC_AC-Clg_Coil> a brick:Cooling_Coil .

<urn:bldg/Core_ZN-PSC_AC-Damper> a brick:Damper .

<urn:bldg/Core_ZN-PSC_AC-Fan> a brick:Supply_Fan,
        vav48:sa-fan ;
    brick:hasPoint <urn:bldg/Core_ZN-PSC_AC-Fanp4>,
        <urn:bldg/Core_ZN-PSC_AC-Fanp5>,
        <urn:bldg/Core_ZN-PSC_AC-Fanp6> .

<urn:bldg/Core_ZN-PSC_AC-Fanp4> a brick:Start_Stop_Command .

<urn:bldg/Core_ZN-PSC_AC-Fanp5> a brick:Fan_Status .

<urn:bldg/Core_ZN-PSC_AC-Fanp6> a brick:Frequency_Command .

<urn:bldg/Core_ZN-PSC_AC-Htg_Coil> a brick:Heating_Coil .
""")

In [57]:
# load the "updated_BRICK_model_you_just_created.ttl"
model = Model.create(BLDG, description="This is a test model for a simple building")
model.graph.parse("updated_BRICK_model_you_just_created.ttl", format="ttl")

<Graph identifier=c9814f3c-6bbe-47e2-b396-4e4e85f81bf5 (<class 'rdflib.graph.Graph'>)>

In [58]:
# display validation results in more detail

validation_result = model.validate(all_shape_graphs)
print(f"Model is valid? {validation_result.valid}")
print("-"*79) # just a separator for better error display
print(validation_result.report_string)
print("-"*79) # just a separator for better error display
print("Model is invalid for these reasons:")
for diff in validation_result.diffset:
    print(f" - {diff.reason()}")

Model is valid? True
-------------------------------------------------------------------------------
Validation Report
Conforms: True

-------------------------------------------------------------------------------
Model is invalid for these reasons:
